In [ ]:
# Colab에서 KoBERT 최신 버전 정보 입력
!pip install gluonnlp pandas tqdm   
!pip install mxnet
!pip install sentencepiece==0.1.91
!pip install transformers==4.8.2
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp310-cp310-linux_x86_64.whl size=681065 sha256=2cf1e3f560567a0d6ec140e12f45f70e202449b5c94a9483d95aaea45ef918c3
  Stored in directory: /root/.cache/pip/wheels/1a/1e/0d/99f55911d90f2b95b9f7c176d5813ef3622894a4b30fde6bd3
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 14.6 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## KoBERT finetuning

In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-9u6086lg
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-9u6086lg
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 MB 12.1 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement onnxruntime<=1.8.0,==1.8.0 (from kobert) (from versions: 1.12.0, 1.12.1, 1.13.1, 1.14.0, 1.14.1)
ERROR: No matching distribution found for onnxruntime<=1.8.0,==1.8.0


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm.notebook import tqdm

In [ ]:
import gluonnlp as nlp

ImportError: ignored

In [ ]:
from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model

ModuleNotFoundError: ignored

In [ ]:
from transformers import AdamW #Adam의 BERT 버전
from transformers.optimization import get_cosine_schedule_with_warmup

ModuleNotFoundError: ignored

In [ ]:
## GPU
device = torch.device("cuda:0")

In [ ]:
from google.colab import drive
drive.mount('/content/drive') 

In [ ]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

## 데이터셋 로드 및 매핑

In [ ]:
import pandas as pd
data = pd.read_excel('/content/drive/MyDrive/한국정보과학회-2023/최종 데이터(수정).xlsx')

In [ ]:
data.columns #고장원인이 input, 위험도등급이 5

Index(['Unnamed: 0', '호기', '부위', '일자', '시작시간', '완료시간', '정지시간(분)', '정지시간(시간)',
       '고장 원인', '처리 내용', 'risk_impact', 'risk_likehood', '위험도 수준', '위험도 등급'],
      dtype='object')

In [ ]:
data = data.dropna()

In [ ]:
data

,Unnamed: 0,호기,부위,일자,시작시간,완료시간,정지시간(분),정지시간(시간),고장 원인,처리 내용,risk_impact,risk_likehood,위험도 수준,위험도 등급
0,0,1호기,압출기,3월28일,18:00:00,18:40:00,40.0,0.67,메인 및 부스타 펌프 트립,작업자 부주위로 노커 케이불 누전,3.426264,15.0,저,5
1,1,1호기,압출기,4월11일,13:45:00,17:17:00,212.0,3.53,콘테이너 메인배관 불럭,풀렌지 볼트 전단 씰 및 볼트 교체,3.426264,15.0,저,5
2,2,1호기,압출기,4월11일,08:30:00,10:43:00,133.0,2.22,빌레트 로더 기계와 추돌,로 인한 피다 휨 교정 작업,3.426264,15.0,저,5
3,3,1호기,압출기,6월23일,10:44:00,12:30:00,106.0,1.77,콘테이너 실린다 고정,실린다 원 위치 조정 작업,3.426264,15.0,저,5
4,4,1호기,압출기,8월13일,01:00:00,01:30:00,30.0,0.50,빌레트 로더 전/후진시 소음 발생,테이너 센터 틀려 조정 작업,3.426264,15.0,저,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
818,818,8호기,후면,2일,11:40:00,12:40:00,60.0,1.00,프르그램 에러,수정작업,0.534613,21.0,중,4
819,819,8호기,후면,19일,17:00:00,17:30:00,30.0,0.50,볼트 파손,교체,0.534613,21.0,중,4
820,820,8호기,기타,2014-09-24 00:00:00,09:00:00,09:15:00,15.0,0.30,op판넬 터치 통신 끊킴,PLC통신 채널 변경 후 정상동작,0.161559,5.0,저,6
822,822,8호기,기타,20일,17:50:00,18:06:00,16.0,0.30,전기실 VCB트립,제투입,0.161559,5.0,저,6


In [ ]:
data.reset_index(drop=True,inplace=True)

In [ ]:
data.loc[(data['위험도 등급'] == 1), '위험도 등급'] = 0  # 1 >> 0
data.loc[(data['위험도 등급'] == 2), '위험도 등급'] = 1  #
data.loc[(data['위험도 등급'] == 3), '위험도 등급'] = 2  #
data.loc[(data['위험도 등급'] == 4), '위험도 등급'] = 3  #
data.loc[(data['위험도 등급'] == 5), '위험도 등급'] = 4  #
data.loc[(data['위험도 등급'] == 6), '위험도 등급'] = 5  # 6 >> 5


data_list = []
for ques, label in zip(data['고장 원인'], data['위험도 등급'])  :
    data = []   
    data.append(ques)
    data.append(str(label))

    data_list.append(data)

In [ ]:
data_list[3:9]

[['콘테이너 실린다 고정 ', '4'],
 ['빌레트 로더 전/후진시 소음 발생', '4'],
 ['콤푸레샤100HP  고장 물 유입', '4'],
 ['OP판넬에서 콘테이너히터#1,2 온도 표시 불능', '4'],
 ['콘테이너 실린더 씰링 안됨', '4'],
 ['콘테이너 씰링 안됨', '4']]

## Train, Test data split

In [ ]:
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.3, random_state=42)

In [ ]:
print(len(dataset_train))
print(len(dataset_test))
#좀 너무 작긴하네....

527
226


## KoBERT 입력데이터 변환 및 파라미터 세팅

각각 토큰화, 정수 인코딩, 패딩 등 적용한다.

Bert는 Input = Token Embeddings + Segment Embeddings + Position Embeddings로 이루어진다.

위의 작업을 편하게 진행해주는 함수: BERTSentenceTransform

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

#인코더 부분. BERTSentenceTransform은

In [ ]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
# 토큰화 및 dataloader 적용
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

# 기존에 train_test_split으로 나눈 dataset_train, dataset_test를 BERTDataset를 통해 tokenization, int encoding, padding 진행
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

#torch 형식의 dataset 처리
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
data_train[0] #토큰화와 패딩이 잘 이루어져 있나

(array([   2, 3647, 7240, 7659, 3036, 6134, 5743, 7063,    3,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(9, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 2)

3개의 array 출력
1. 패딩된 시퀀스
2. 길이와 타입에 관한 내용
3. attention mask 데이터

## KoBert 학습모델 만들기

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=6,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
 
#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 대표적인 loss func

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)


#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

## KoBERT 학습시키기

In [ ]:
from tqdm import tqdm, tqdm_notebook

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


        optimizer.zero_grad()
        
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-36-ff7e6a9dcebe>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/9 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.860363483428955 train acc 0.140625
epoch 1 train acc 0.21585648148148148


<ipython-input-36-ff7e6a9dcebe>:27: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/4 [00:00<?, ?it/s]

...

epoch 9 test acc 0.41681985294117646


  0%|          | 0/9 [00:00<?, ?it/s]

epoch 10 batch id 1 loss 1.153534173965454 train acc 0.59375
epoch 10 train acc 0.5519675925925926


  0%|          | 0/4 [00:00<?, ?it/s]

epoch 10 test acc 0.3977481617647059


In [ ]:
def softmax(a) :
    exp_a = np.exp(a)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    
    return y

## 새로운 문장 테스트

In [ ]:
data_list

[['메인 및 부스타 펌프 트립', '4'],
 ['콘테이너 메인배관 불럭', '4'],
 ['빌레트 로더 기계와 추돌', '4'],
 ['콘테이너 실린다 고정 ', '4'],
 ['빌레트 로더 전/후진시 소음 발생', '4'],
 ['콤푸레샤100HP  고장 물 유입', '4'],
 ['OP판넬에서 콘테이너히터#1,2 온도 표시 불능', '4'],
 ['콘테이너 실린더 씰링 안됨', '4'],
 ['콘테이너 씰링 안됨', '4'],
 ['콘테이너 배관 누유', '4'],
 ['유압배관 크랙으로 인한 누유현상', '4'],
 ['냉각수 펌프 교체작업', '4'],
 ['메인펌프 자동수동 on 불능', '4'],
 ['이젝트 후진동작 불능', '4'],
 ['푸샤 엔코더 베아링 마모', '4'],
 ['케이불 열화로 단선', '4'],
 ['빌레트 이송 체인 전단', '4'],
 ['프르브 열전대 에러', '4'],
 ['빌레트 가열로 START不', '4'],
 ['프르브 온도 감지 에러', '4'],
 ['ZONE#1 파이로트 점화 불능', '4'],
 ['탑 클램프 실린더 누유', '4'],
 ['푸샤 체인 아타치 마모', '4'],
 ['빌레트가열로(1호기) 터치스크린 버튼 안눌림', '4'],
 ['리턴 감지센서 불능', '4'],
 ['빌레트 감지센서 파손', '4'],
 ['이젝트 공급 센서 감지 불능', '4'],
 ['풀러 2호 체인 고정 볼트 ', '5'],
 ['Hi-temp C/V 수동 구동은 되나 자동 불능', '4'],
 ['파워헤드업/다운 S/W 소손', '4'],
 ['tail head jaw 하강 동작 불능', '4'],
 ['교정기 모니터 안됨.', '4'],
 ['교정기 테일헤드 조 하강 안됨 조사이 알루미늄 끼임', '4'],
 ['비디오 모니터 화면 불능', '4'],
 ['교정기-1호 CCTV MONITOR 화면 이상으로 점검', '4'],
 ['자동 운전시 GAUGE HAED UP동작후 C/V 구동 불능', '4'],
 

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()
            softmax_lgt = list(softmax(logits))

        mapping = {}
            


        for i, v in enumerate(softmax_lgt, start=1):
          mapping[v] = i
          
        ls_sorted_keys = []

        for k in sorted(mapping.keys(), reverse=True):
          v = mapping[k]
          ls_sorted_keys.append("해당 고장은 위험도 {}등급이 나올 확률이 {:.3f}%입니다. 해당 요인도를 참고 바랍니다".format(v,k))

          if len(ls_sorted_keys) == 3:
            break



    return ls_sorted_keys
    

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
predict('콘테이너 실린더 누유 및 파손')

['해당 고장은 위험도 3등급이 나올 확률이 0.454%입니다. 해당 요인도를 참고 바랍니다',
 '해당 고장은 위험도 4등급이 나올 확률이 0.314%입니다. 해당 요인도를 참고 바랍니다',
 '해당 고장은 위험도 1등급이 나올 확률이 0.077%입니다. 해당 요인도를 참고 바랍니다']

# LDA 처리과정